# NYC Apartment Search

_[Project prompt](https://docs.google.com/document/d/1BYVyFBDcTywdUlanH0ysfOrNWPgl7UkqXA7NeewTzxA/edit#heading=h.bpxu7uvknnbk)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an idea of a possible approach.**_

## Setup

In [1]:
pip install geoalchemy2 geopandas shapely psycopg2 psycopg2-binary sqlalchemy matplotlib requests


Defaulting to user installation because normal site-packages is not writeable
  Using cached psycopg2-2.9.9.tar.gz (384 kB)
     |████████████████████████████████| 2.8 MB 7.9 MB/s eta 0:00:01
  Created wheel for psycopg2: filename=psycopg2-2.9.9-cp39-cp39-macosx_10_9_universal2.whl size=244983 sha256=d8ca854ed0f31e9bd88df3b581b360d7932b4b1807133f650b083c3e8ebb643b
  Stored in directory: /Users/mac/Library/Caches/pip/wheels/3a/06/25/adb124afd8c8346e45c455f6586f7289cde2b4e339dfbcd9e9
Successfully built psycopg2
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# All import statements needed for the project, for example:

import json
import pathlib
import urllib.parse

import geoalchemy2 as gdb
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import requests
import shapely
import sqlalchemy as db

from sqlalchemy.orm import declarative_base

/Users/mac/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
# Any constants you might need; some have been added for you

# Where data files will be read from/written to - this should already exist
DATA_DIR = pathlib.Path(".")
ZIPCODE_DATA_FILE = DATA_DIR / "zipcodes" / "nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

NYC_DATA_APP_TOKEN = "8ITaLVGKJEzelLCfrNyuIi2rJ"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/resource/"
NYC_DATA_311 = "erm2-nwe9.geojson"
NYC_DATA_TREES = "5rq2-4hqu.geojson"

DB_NAME = "Project"
DB_USER = "yawen"
DB_URL = f"postgresql+psycopg2://{DB_USER}@localhost/{DB_NAME}"
DB_SCHEMA_FILE = "schema.sql"
# directory where DB queries for Part 3 will be saved
QUERY_DIR = pathlib.Path("queries")

In [5]:
# Make sure the QUERY_DIRECTORY exists
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()

## Part 1: Data Preprocessing

In [6]:
def download_nyc_geojson_data(url, force=False):
    parsed_url = urllib.parse.urlparse(url)
    url_path = parsed_url.path.strip("/")
    
    filename = DATA_DIR / (url_path.split('/')[-1])
    
    if force or not filename.exists():
        print(f"Downloading {url} to {filename}...")
        response = requests.get(url)
        response.raise_for_status()
        with open(filename, "w") as f:
            json.dump(response.json(), f)
        print(f"Done downloading {url}.")

    else:
        print(f"Reading from {filename}...")

    return filename

In [7]:
def load_and_clean_zipcodes(zipcode_datafile):
    
    gdf = gpd.read_file(zipcode_datafile)
    
    
    #normalized data
    gdf_normalized = gdf.to_crs(epsg=4326)
    
    #selected_zipcode
    columns=['ZIPCODE', 'PO_NAME', 'STATE','COUNTY','geometry']
    gdf_selected=gdf_normalized[columns]
    
    #cleaned_zipcode
    gdf_cleaned = gdf_selected.dropna()
    
    return gdf_cleaned

In [8]:
def download_and_clean_311_data():
    url = f"{BASE_NYC_DATA_URL}{NYC_DATA_311}?$$app_token={NYC_DATA_APP_TOKEN}&$limit=1000"
    
    filename = download_nyc_geojson_data(url)
    
    df = gpd.read_file(filename)
    columns = ['unique_key', 'created_date', 'incident_zip', 'complaint_type']
    df_selected = df[columns]
    df_cleaned = df_selected.dropna()

    return df_cleaned

In [9]:
def download_and_clean_tree_data():
    url = f"{BASE_NYC_DATA_URL}{NYC_DATA_TREES}?$$app_token={NYC_DATA_APP_TOKEN}&$limit=1000"
    
    filename = download_nyc_geojson_data(url)
    
    df = gpd.read_file(filename)
    columns = ['tree_id','zipcode','x_sp', 'y_sp','longitude', 'latitude','spc_common','health','status']
    df_selected = df[columns]
    df_cleaned = df_selected.dropna()

    return df_cleaned
    
    


In [11]:
def load_and_clean_zillow_data():
    df = pd.read_csv(ZILLOW_DATA_FILE)
    df_cleaned = df.dropna()
    return df_cleaned

In [10]:
def load_all_data():
    geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)
    geodf_311_data = download_and_clean_311_data()
    geodf_tree_data = download_and_clean_tree_data()
    df_zillow_data = load_and_clean_zillow_data()
    return (
        geodf_zipcode_data,
        geodf_311_data,
        geodf_tree_data,
        df_zillow_data
    )

In [11]:
geodf_zipcode_data, geodf_311_data, geodf_tree_data, df_zillow_data = load_all_data()

DriverError: zipcodes/nyc_zipcodes.shp: No such file or directory

In [11]:
# Show basic info about each dataframe
geodf_zipcode_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ZIPCODE     263 non-null    object  
 1   BLDGZIP     263 non-null    object  
 2   PO_NAME     263 non-null    object  
 3   POPULATION  263 non-null    float64 
 4   AREA        263 non-null    float64 
 5   STATE       263 non-null    object  
 6   COUNTY      263 non-null    object  
 7   ST_FIPS     263 non-null    object  
 8   CTY_FIPS    263 non-null    object  
 9   URL         263 non-null    object  
 10  SHAPE_AREA  263 non-null    float64 
 11  SHAPE_LEN   263 non-null    float64 
 12  geometry    263 non-null    geometry
dtypes: float64(4), geometry(1), object(8)
memory usage: 26.8+ KB


In [12]:
# Show first 5 entries about each dataframe
geodf_zipcode_data.head()

,ZIPCODE,BLDGZIP,PO_NAME,POPULATION,AREA,STATE,COUNTY,ST_FIPS,CTY_FIPS,URL,SHAPE_AREA,SHAPE_LEN,geometry
0,11436,0,Jamaica,18681.0,2.269930e+07,NY,Queens,36,081,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.80585 40.68291, -73.80569 40.682..."
1,11213,0,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.93740 40.67973, -73.93487 40.679..."
2,11212,0,Brooklyn,83866.0,4.197210e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.90294 40.67084, -73.90223 40.668..."
3,11225,0,Brooklyn,56527.0,2.369863e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.95797 40.67066, -73.95576 40.670..."
4,11218,0,Brooklyn,72280.0,3.686880e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.97208 40.65060, -73.97192 40.650..."


In [13]:
geodf_311_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 997 entries, 0 to 999
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   unique_key      997 non-null    object        
 1   created_date    997 non-null    datetime64[ns]
 2   incident_zip    997 non-null    object        
 3   complaint_type  997 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 38.9+ KB


In [14]:
geodf_311_data.head()

,unique_key,created_date,incident_zip,complaint_type
0,59551848,2023-11-27 12:00:00,11370,Derelict Vehicles
1,59551851,2023-11-27 12:00:00,10468,Derelict Vehicles
2,59551328,2023-11-27 00:56:00,11217,Traffic Signal Condition
3,59554861,2023-11-27 00:41:00,11429,Adopt-A-Basket
4,59553569,2023-11-27 00:34:00,10474,Traffic Signal Condition


In [15]:
geodf_tree_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 973 entries, 0 to 999
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tree_id     973 non-null    object
 1   zipcode     973 non-null    object
 2   x_sp        973 non-null    object
 3   y_sp        973 non-null    object
 4   longitude   973 non-null    object
 5   latitude    973 non-null    object
 6   spc_common  973 non-null    object
 7   health      973 non-null    object
 8   status      973 non-null    object
dtypes: object(9)
memory usage: 76.0+ KB


In [16]:
geodf_tree_data.head()

,tree_id,zipcode,x_sp,y_sp,longitude,latitude,spc_common,health,status
0,180683,11375,1027431.14821,202756.768749,-73.84421522,40.72309177,red maple,Fair,Alive
1,200540,11357,1034455.70109,228644.837379,-73.81867946,40.79411067,pin oak,Fair,Alive
2,204026,11211,1001822.83131,200716.891267,-73.9366077,40.71758074,honeylocust,Good,Alive
3,204337,11211,1002420.35833,199244.253136,-73.93445616,40.71353749,honeylocust,Good,Alive
4,189565,11215,990913.775046,182202.425999,-73.97597938,40.66677776,American linden,Good,Alive


In [17]:
df_zillow_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 585 entries, 0 to 6556
Columns: 114 entries, RegionID to 2023-09-30
dtypes: float64(105), int64(3), object(6)
memory usage: 525.6+ KB


In [18]:
df_zillow_data.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2015-01-31,...,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30
0,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,1606.206406,...,1994.653463,2027.438438,2042.237444,2049.325559,2016.531345,2023.438976,2031.558202,2046.144009,2053.486247,2055.771355
1,91940,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,1257.814660,...,1749.697900,1738.217986,1747.305840,1758.407295,1758.891075,1762.980879,1771.751591,1779.338402,1795.384582,1799.631140
8,91926,11,77433,zip,TX,TX,Cypress,"Houston-The Woodlands-Sugar Land, TX",Harris County,1332.384333,...,1881.204550,1885.695935,1884.894986,1880.532012,1870.035369,1863.111029,1892.511066,1922.759295,1945.581823,1975.672556
15,62037,19,11226,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Kings County,1944.609891,...,2695.164727,2680.683700,2676.791284,2697.414702,2706.080701,2726.381017,2753.966159,2770.403005,2785.320137,2762.435713
17,70829,21,30044,zip,GA,GA,Lawrenceville,"Atlanta-Sandy Springs-Alpharetta, GA",Gwinnett County,1157.569878,...,2125.698496,2113.964250,2170.556028,2160.008767,2191.894968,2181.780560,2161.711011,2117.990720,2119.319392,2055.476496


## Part 2: Storing Data

In [12]:
def setup_new_postgis_database(DB_NAME,DB_USER,DB_SCHEMA_FILE):


    !createdb {DB_NAME}
    !psql --dbname {DB_NAME} -c 'CREATE EXTENSION postgis;'
    !psql -d {DB_NAME} -U {DB_USER}
    
    !psql --dbname {DB_NAME} -U {DB_USER} -f {DB_SCHEMA_FILE}


In [14]:
setup_new_postgis_database(DB_NAME,DB_USER,DB_SCHEMA_FILE)

CREATE EXTENSION
psql (16.1)
Type "help" for help.

Project=> 

### Creating Tables


These are just a couple of options to creating your tables; you can use one or the other, a different method, or a combination.

In [ ]:
engine = db.create_engine(DB_URL)

#### Option 1: SQL

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the SQL statements to create your 4 tables

ZIPCODE_SCHEMA = """CREATE TABLE IF NOT EXISTS zipcode
(ZIPCODE SERIAL PRIMARY KEY,
    PO_NAME CHAR(50),
    STATE CHAR(50),
    COUNTY CHAR(50),
    geometry GEOMETRY(Point, 4326)
);

"""


NYC_311_SCHEMA = """
CREATE TABLE IF NOT EXISTS nyc_311
(unique_key SERIAL PRIMARY KEY,
    created_date DATE,
    incident_zip CHAR(50),
    complaint_type CHAR(50)
);

"""

NYC_TREE_SCHEMA = """CREATE TABLE IF NOT EXISTS nyc_tree
(tree_id SERIAL PRIMARY KEY,
    zipcode CHAR(50),
    x_sp CHAR(50),
    y_sp CHAR(50),
    longtitude CHAR(50),
    latitude CHAR(50),
    spc_common CHAR(50),
    health CHAR(50),
    status CHAR(50)
);

"""

ZILLOW_SCHEMA = """CREATE TABLE IF NOT EXISTS zillow
(region_id SERIAL PRIMARY KEY,
    region_name CHAR(50),
    SizeRank INTEGER,
    Region_type CHAR(50),
    StateName CHAR(50),
    State CHAR(50),
    City CHAR(50),
    Metro CHAR(50),
    CountryName CHAR(50),
    2015-01-31 FLOAT,
    2015-02-28 FLOAT, 
    2015-03-31 FLOAT, 
    2015-04-30 FLOAT,
    2015-05-31 FLOAT,
    2015-06-30 FLOAT,
    2015-07-31 FLOAT,
    2015-08-31 FLOAT, 
    2015-09-30 FLOAT,
    2015-10-31 FLOAT, 
    2015-11-30 FLOAT,
    2015-12-31 FLOAT,
    2016-01-31 FLOAT,
    2016-02-29 FLOAT,
    2016-03-31 FLOAT, 
    2016-04-30 FLOAT,
    2016-05-31 FLOAT,
    2016-06-30 FLOAT,
    2016-07-31 FLOAT, 
    2016-08-31 FLOAT,
    2016-09-30 FLOAT,
    2016-10-31 FLOAT, 
    2016-11-30 FLOAT, 
    2016-12-31 FLOAT,
    2017-01-31 FLOAT,
    2017-02-28 FLOAT, 
    2017-03-31 FLOAT, 
    2017-04-30 FLOAT,
    2017-05-31 FLOAT,
    2017-06-30 FLOAT,
    2017-07-31 FLOAT,
    2017-08-31 FLOAT,
    2017-09-30 FLOAT, 
    2017-10-31 FLOAT,
    2017-11-30 FLOAT,
    2017-12-31 FLOAT,
    2018-01-31 FLOAT, 
    2018-02-28 FLOAT,
    2018-03-31 FLOAT,
    2018-04-30 FLOAT,
    2018-05-31 FLOAT,
    2018-06-30 FLOAT, 
    2018-07-31 FLOAT,
    2018-08-31 FLOAT, 
    2018-09-30 FLOAT,
    2018-10-31 FLOAT, 
    2018-11-30 FLOAT,
    2018-12-31 FLOAT,
    2019-01-31 FLOAT, 
    2019-02-28 FLOAT,
    2019-03-31 FLOAT, 
    2019-04-30 FLOAT,
    2019-05-31 FLOAT, 
    2019-06-30 FLOAT,
    2019-07-31 FLOAT,
    2019-08-31 FLOAT,
    2019-09-30 FLOAT,
    2019-10-31 FLOAT,
    2019-11-30 FLOAT, 
    2019-12-31 FLOAT,
    2020-01-31 FLOAT, 
    2020-02-29 FLOAT,
    2020-03-31 FLOAT,
    2020-04-30 FLOAT, 
    2020-05-31 FLOAT,
    2020-06-30 FLOAT,
    2020-07-31 FLOAT, 
    2020-08-31 FLOAT, 
    2020-09-30 FLOAT,
    2020-10-31 FLOAT,
    2020-11-30 FLOAT,
    2020-12-31 FLOAT,
    2021-01-31 FLOAT,
    2021-02-28 FLOAT, 
    2021-03-31 FLOAT, 
    2021-04-30 FLOAT,
    2021-05-31 FLOAT,
    2021-06-30 FLOAT,
    2021-07-31 FLOAT,
    2021-08-31 FLOAT,
    2021-09-30 FLOAT,
    2021-10-31 FLOAT, 
    2021-11-30 FLOAT, 
    2021-12-31 FLOAT,
    2022-01-31 FLOAT,
    2022-02-28 FLOAT, 
    2022-03-31 FLOAT, 
    2022-04-30 FLOAT, 
    2022-05-31 FLOAT, 
    2022-06-30 FLOAT, 
    2022-07-31 FLOAT, 
    2022-08-31 FLOAT,
    2022-09-30 FLOAT, 
    2022-10-31 FLOAT,
    2022-11-30 FLOAT, 
    2022-12-31 FLOAT,
    2023-01-31 FLOAT,
    2023-02-28 FLOAT, 
    2023-03-31 FLOAT,
    2023-04-30 FLOAT,
    2023-05-31 FLOAT
    2023-06-30 FLOAT,
    2023-07-31 FLOAT,
    2023-08-31 FLOAT, 
    2023-09-30 FLOAT,
   
);
"""
indance='''

CREATE INDEX idx_geometries_geom_gist ON zipcode USING gist (geometry);

'''

In [ ]:
# create that required schema.sql file
with open(DB_SCHEMA_FILE, "w") as f:
    f.write(ZIPCODE_SCHEMA)
    f.write(NYC_311_SCHEMA)
    f.write(NYC_TREE_SCHEMA)
    f.write(ZILLOW_SCHEMA)

In [3]:
# If using SQL (as opposed to SQLAlchemy), execute the schema files to create tables
import psycopg2

conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    host='localhost',
    port='5432'
)

cur = conn.cursor()

setup_new_postgis_database(DB_NAME,DB_USER,"schema.sql")

with open(DB_SCHEMA_FILE, 'r') as file:
    sql_commands = file.read()


commands = sql_commands.split(';')


for command in commands:
        if command.strip(): 
            cur.execute(command)

NameError: name 'engine' is not defined

#### Option 2: SQLAlchemy

In [ ]:
Base = declarative_base()

class Tree(Base):
    __tablename__ = "trees"

    ...


In [ ]:
Base.metadata.create_all(engine)

### Add Data to Database

These are just a couple of options to write data to your tables; you can use one or the other, a different method, or a combination.

#### Option 1: SQL

In [ ]:
def write_dataframes_to_table():
   
    engine = create_engine(DB_URL)
    geodf_zipcode_data.to_postgis('zipcode', engine, if_exists='append', index=False)
    geodf_311_data.to_postgis('nyc_311', engine, if_exists='append', index=False)
    geodf_tree_data.to_postgis('nyc_tree', engine, if_exists='append', index=False)
    df_zillow_data .to_postgis('zillow', engine, if_exists='append', index=False)
    


In [ ]:
write_dataframes_to_table()

#### Option 2: SQLAlchemy

In [ ]:
Session = db.orm.sessionmaker(bind=engine)
session = Session()

In [ ]:
for row in geodf_tree_data.iterrows():
    tree = Tree(...)
    session.add(tree)

In [ ]:
session.commit()

## Part 3: Understanding the Data

### Query 1

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

In [ ]:
QUERY_1_FILENAME = QUERY_DIR / "FILL_ME_IN"

QUERY_1 = """
FILL_ME_IN
"""

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_1))
    for row in result:
        print(row)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query your database for the data needed.
    # You can put the data queried into a pandas/geopandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)